# Introduction

Semantic search means search with meaning or context. It is useful in sentence/phrase matching application at scale. Usually, Inference is requested for single sentence or in batches but, it still doesn't beat the purpose of matching a phrase with existing corpus of million (let's say) documents.

Just imagine, How troublesome that inference would be! That's why, in this article, we are going to implement a semantic search using HuggingFace [sentence-transformers](https://www.sbert.net/) model ([all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)) and [FAISS](https://pypi.org/project/faiss/)


**About FIASS**

Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Faiss is written in C++ with complete wrappers for Python/numpy. Some of the most useful algorithms are implemented on the GPU. It is developed by Facebook AI Research.

# Implementation

## 1. Installed required libraries

In [ ]:
!pip install faiss-cpu
!pip install -U sentence-transformers

## 2. Imports

In [ ]:
import numpy as np
import torch
import os
import pandas as pd
import faiss
import time
from sentence_transformers import SentenceTransformer

## 3. Create or Load a corpus

In [ ]:
documents = [
    "Pet Store",
    "food store",
    "cat Store"
]

## 4. Loading the pre-trained model

[Usage Guide](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2#usage-sentence-transformers) for reference

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
model

In [ ]:
encoded_data = model.encode(documents)

## 5. Create index for your corpus
[Guidelines to choose an index](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index). Create index and add your corpus/data to it.

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(model.get_sentence_embedding_dimension()))
index.add_with_ids(encoded_data, np.array(range(0, len(documents))))

In [ ]:
# Serializing Index to export it across different host
faiss.write_index(index, 'sample_documents')
# De-serializing the index
index = faiss.read_index('sample_documents')

## 6. Perform semantic similarity

In [ ]:
def search(query):
    t=time.time()
    query_vector = model.encode([query])
    # Search for top k results
    k = 1
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [documents[_id] for _id in top_k[1].tolist()[0]]

## 7. Search from index

In [ ]:
query="Dog store"
results=search(query)
print('results :')
for result in results:
    print('\t',result)

In [ ]:
nan